In [1]:
import pandas as pd
import os

In [2]:
model_ckpt_dict = {
    'bloom-560m': [1000, 10000, 100000, 200000, 300000, 400000, 500000, 600000],
    'bloom-1b1': [1000, 10000, 100000, 200000, 300000, 400000, 500000, 600000],
    'bloom-1b7': [1000, 10000, 50000, 100000, 150000, 200000, 250000, 300000], 
}

trg_langs = ['ar', 'es', 'eu', 'fr', 'hi', 'pt', 'ta', 'ur', 'vi']

similarity_result_path = os.path.join('../parallel-sentence-similarity/experiments/cos_similarity_csv/')
output_csv_file_path = 'csv_files/'
layer = 'last-layer'

In [3]:
def read_similarity_by_langs(path, lang_list, read_code=False):
    similarity_dict = {}
    for lang in lang_list:
        result_file = os.path.join(path, f'en-{lang}.csv')
        with open(result_file, 'r') as f:
            last_line = f.readlines()[-1]
            similarity_dict[lang] = float(last_line.split(',')[1])
    if read_code:
        result_file = os.path.join(path, f'nl-code.csv')
        with open(result_file, 'r') as f:
            last_line = f.readlines()[-1]
            similarity_dict['code'] = float(last_line.split(',')[1])
    return similarity_dict

In [4]:
for model, ckpt_list in model_ckpt_dict.items():

    output_csv_file_name = os.path.join(output_csv_file_path, f'{model}_{layer}_cos-similarity.csv')
    ckpt_similarity_dict = {}
    # Dict{ckpt: Dict{ lang: similarity}}
    ckpt_similarity_dict['best'] = read_similarity_by_langs(os.path.join(similarity_result_path, model, layer), trg_langs, read_code=True)
    
    for ckpt in ckpt_list:
        ckpt_similarity_dict[ckpt] = read_similarity_by_langs(os.path.join(similarity_result_path, f'{model}-intermediate-global_step{ckpt}', layer), trg_langs, read_code=True)
    
    df = pd.DataFrame.from_dict(ckpt_similarity_dict)
    print(df)
    df.to_csv(output_csv_file_name)




          best      1000     10000    100000    200000    300000    400000  \
ar    0.077330  0.457647  0.203364  0.176615  0.027245  0.678346  0.105271   
es    0.887743  0.727895  0.826337  0.889176  0.990906  0.993016  0.996312   
eu    0.746171  0.703451  0.644629  0.765071  0.769899  0.927408  0.957210   
fr    0.894948  0.727756  0.828873  0.905522  0.992900  0.995207  0.996782   
hi    0.116905  0.657520  0.286740  0.358499  0.062266  0.669454 -0.015551   
pt    0.902158  0.760800  0.795679  0.875740  0.989301  0.990275  0.997125   
ta    0.088106  0.577768  0.201776  0.226207  0.001060  0.158025  0.048556   
ur    0.129696  0.466826  0.146380  0.232983 -0.143666  0.574029  0.228214   
vi    0.865738  0.665003  0.490437  0.753974  0.928092  0.941562  0.940514   
code  0.953850  0.135067  0.376788  0.518225  0.984069  0.990153  0.981627   

        500000    600000  
ar    0.203364  0.356799  
es    0.826337  0.734460  
eu    0.644629  0.746231  
fr    0.828873  0.737500  
hi    